# LinkedIn Profile Analyzer

This Colab notebook is designed to analyze LinkedIn profiles by fetching user details and posts via LinkedIn's API. It then performs a detailed analysis using a custom AI model hosted by OpenRouter.

## Purpose

The purpose of this notebook is to provide insights into the professional background, engagement levels, and areas of expertise of LinkedIn users, which can be particularly useful for creatives, professionals, marketers, or personal branding experts.

## How It Works

1. The notebook retrieves user details and their latest posts from LinkedIn.
2. It constructs a comprehensive prompt with these details and asks an AI model to analyze the content.
3. The AI model evaluates the posts to provide insights on professional interests, skills, trends in discussion topics, and recommendations for professional growth.

## API Key and Sensitive Data

- **OPENROUTER_API_KEY**: This is a placeholder for the actual API key used to authenticate requests to OpenRouter's AI service.
- **OPEN-AI-4_API_KEY**
- **Rapid API Linkedin Key**: https://rapidapi.com/freshdata-freshdata-default/api/fresh-linkedin-profile-data

## Setup Instructions

1. Replace `OPENROUTER_API_KEY` with your actual OpenRouter API key.
2. Replace `OPEN_AI_KEY` with your actual OpenAI-4 API key.
3. Replace Rapid API key placeholders with your actual LinkedIn API keys.
4. Enter the URL of the LinkedIn profile you wish to analyze when prompted.

## Precautions

- Be mindful of API usage limits and potential costs associated with high usage.
- Handle all personal data fetched from LinkedIn responsibly, adhering to data privacy laws and LinkedIn's terms of service.

## Disclaimer

This tool is intended for educational and professional purposes only. Always ensure you have permission to analyze a LinkedIn profile, especially if using the data for commercial purposes.


In [3]:
import http.client
import urllib.parse
import json
import requests

# Constants
OPENROUTER_API_KEY = "your_api_key"
OPEN_AI_KEY="your_api_key"
YOUR_SITE_URL = "https://jeezai.com"
YOUR_APP_NAME = "JeezAI"

def get_linkedin_user_details(linkedin_url):
    encoded_url = urllib.parse.quote(linkedin_url, safe='')
    conn = http.client.HTTPSConnection("fresh-linkedin-profile-data.p.rapidapi.com")
    headers = {
        'X-RapidAPI-Key': "your_api_key",
        'X-RapidAPI-Host': "fresh-linkedin-profile-data.p.rapidapi.com"
    }
    conn.request("GET", f"/get-linkedin-profile?linkedin_url={encoded_url}&include_skills=false", headers=headers)
    res = conn.getresponse()
    data = res.read().decode("utf-8")
    return json.loads(data)

def get_linkedin_posts(linkedin_url):
    encoded_url = urllib.parse.quote(linkedin_url, safe='')
    conn = http.client.HTTPSConnection("fresh-linkedin-profile-data.p.rapidapi.com")
    headers = {
        'X-RapidAPI-Key': "your_api_key",
        'X-RapidAPI-Host': "fresh-linkedin-profile-data.p.rapidapi.com"
    }
    conn.request("GET", f"/get-profile-posts?linkedin_url={encoded_url}&type=posts", headers=headers)
    res = conn.getresponse()
    data = res.read().decode("utf-8")
    return json.loads(data)

def generate_text(prompt):
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Referer": YOUR_SITE_URL,
        "X-Title": YOUR_APP_NAME,
        "Content-Type": "application/json"
    }
    data = {
        "model": "cohere/command-r-plus",
        "max_tokens": 4000,
        "temperature": 0,
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=data)
    return response.json()

def generate_career(prompt):
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "x-api-key": f"{OPEN_AI_KEY}",
        "Referer": YOUR_SITE_URL,
        "X-Title": YOUR_APP_NAME,
        "Content-Type": "application/json"
    }
    data = {
        "model": "openai/gpt-4-turbo",
        "max_tokens": 4000,
        "temperature": 0,
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=data)
    return response.json()



In [4]:
linkedin_url = input("Please enter the LinkedIn profile URL: ")
target_career=input("Please enter the Target Career Role you want to achieve:")
user_details = get_linkedin_user_details(linkedin_url)
user_details_no_urls = {key: value for key, value in user_details['data'].items() if not isinstance(value, str) or 'http' not in value}
user_details_string = json.dumps(user_details_no_urls)
posts_data = get_linkedin_posts(linkedin_url)
posts = posts_data.get('data', [])


# Display the posts
print("\nExtracted Posts:")
for i, post in enumerate(posts, 1):
    if 'text' in post:
        print(f"Post {i}: {post['text']}")
    else:
        print(f"Post {i}: [No text available]")

if posts:
    prompt = f"""
    LinkedIn Profile Analysis

    User Summary:
    Change the below things accorinding to the user data,the below is just an example:
    - Name: John Doe
    - Profile Summary: Experienced AI professional with a passion for innovation.
    - Current Role: AI Researcher at Tech Innovations Inc.
    - Profile URL: {linkedin_url}
    - Education: Bachelor of Science in Computer Science, University of Tech, Master of AI, AI Institute
    - Experiences: AI Engineer at AI Solutions Co., Data Scientist at Data Insights Ltd(Please Try to get all the previous experience of the user and list them)
    - Interests: AI research, machine learning, data science

    Atleast write about 200 words for each of the following .First write the User Summary according to the user.

    Detailed Analysis Request:
    1. Analyze the technical content of the user's posts. Highlight any innovative ideas or significant contributions to the field of AI.
    2. Extract key phrases or important sentences that showcase the user's expertise and thought leadership.
    3. Assess the engagement levels of the posts (likes, comments, shares) to gauge influence and reach within the professional network.
    4. Identify any trends in the topics discussed over time and how they align with current industry trends.
    5. Evaluate the user's network growth and interactions to understand their community impact and collaborative efforts.


    Professional Interests:
    - List specific areas of AI and technology the user is interested in, based on post content and interactions.

    Skills & Expertise:
    - Detail technical skills, tools, and methodologies mentioned or implied in the user's posts.

    Professional Goals:
    - Infer potential career aspirations and professional development goals from the user's content and interactions.

    Please structure your response with clear headings and bullet points for each section.
    """
    prompt+=user_details_string
    for i, post in enumerate(posts, 1):
        try:
            prompt += f"\nPost {i}: {post['text']}"
        except KeyError:
            prompt += f"\nPost {i}: [No text available]"

    analysis_results = generate_text(prompt)
    if 'choices' in analysis_results:
        print("\nAnalysis Results:\n")
        analysis_messages = []
        for choice in analysis_results['choices']:
            analysis_messages.append(choice['message']['content'])  # Append each message to the list
            print(choice['message']['content'])

        analysis_results_string = "\n".join(analysis_messages)

    else:
        print("Failed to generate analysis results.(Might be due to Prompt tokens limit exceeded on openrouter api)")
else:
    print("No posts found or no text available in posts.(Might be due to you rapidapi key monthly quota limit.)")

Please enter the LinkedIn profile URL: https://www.linkedin.com/in/anmoldesai001/
Please enter the Target Career Role you want to achieve:AI engineer

Extracted Posts:
Post 1: Just finished the course “Python Essential Training”! in this Time  #pythonprogramminglanguage
Post 2: Last weekend, Out of 230 participants, 213 has successfully completed the #Google BigQuery for Machine Learning workshop on Cloud Platform, which is a 92% success rate. We are overwhelmed with this official statics. Sincere thanks to Neal Lilliott and @Madusha Cooray for their immense support. Kudos to the selfless volunteers who worked tirelessly to make it happen.

Uttam Tripathi Naveen Nigam Kyle Paul Kübra Zengin
Ramkumar Arun Shweta Shukla Prachi Bhatnagar Jimberly C.
Google Developer Group (GDG) Cloud Toronto startGBC

#machinelearning #toronto #cloudcomputing 
#technology #gbc #wtm

Analysis Results:

Here is an analysis of the LinkedIn profile and posts provided: 

## User Summary:
- Name: Anmol Desai
- 

In [5]:
prompt_career=f"""
    You are responsible for creating a detailed career roadmap for this user.

    Profile Report:
    Target Career Role: {target_career}


    Create a career report for this user in this structure:/n

    Personalized Career Plan
    Career Path: (By analyzing user's current education and experiences as well as target career roal, create a 10 step plan for the user to launch/upskill their career for the given role. The plan should be comprehensive, technical and challenging. Add 10 steps along with 10 descriptions of each steps)
    Custom Learning Plan: (Create a very technical, assertive and challenging learning plan for the user to launch their career in the given role within 6 months. The plan should be month by month, including different programming languages, hands on and capstone project and networking, skill development and focuses areas particulary for the target goal. You have to personalize this plan by checking user's previous experiences and current knowledge level in the field.)
    Programming Languages to learn: (list the relevant programming languages)
    Books to read: (recommend 10 relevant books with the target goal and their current level.)
    Leaders to follow: (list 10 leaders in this field to follow)

    Below is the complete user detail:
    """

prompt_career+=analysis_results_string

prompt_career+=f"""\n\n Strictly give your responce with clear headings and bullet points for each section."""
career_analysis=generate_career(prompt_career)
if 'choices' in career_analysis:
        print("\nCareer Analysis Results:\n")
        for choice in career_analysis['choices']: # Append each message to the list
            print(choice['message']['content'])



Career Analysis Results:

### Personalized Career Plan

#### Career Path:
1. **Complete Advanced Python Programming Certification**:
   - Enhance your Python skills by completing an advanced course that covers complex topics such as asynchronous programming, decorators, and context managers.

2. **Master Data Structures and Algorithms**:
   - Gain a deep understanding of data structures (like trees, graphs, and hash tables) and algorithms, which are crucial for optimizing AI solutions and passing technical interviews.

3. **Specialize in Machine Learning and Deep Learning**:
   - Enroll in specialized courses to master both theoretical and practical aspects of machine learning and deep learning, focusing on frameworks like TensorFlow and PyTorch.

4. **Develop Expertise in Natural Language Processing (NLP)**:
   - Given your interest in NLP, take advanced courses and work on projects that involve sentiment analysis, chatbots, and machine translation to enhance your skills in this area

## Contributions

Feel free to fork this notebook, suggest improvements, or update the underlying models. If you encounter any issues or have suggestions, please submit a pull request or raise an issue in the repository.